In [ ]:
# cd C:\Users\Venuxk\Projects\panel
# conda activate mypanelintro
# panel serve --show --autoreload elwha.ipynb

from DataMap import DataMap
from ipyleaflet import basemaps
import panel as pn
import os
import pandas as pd
import geopandas
import json
import datetime as dt
pn.extension(sizing_mode="stretch_width")

In [ ]:
# -------------------------------------------------- Constant Variables --------------------------------------------------

# Set base path to data directories.
base_data_path = "data/Elwha/"

# Assign names for map's layer types.
topography_data = "Topography"
bathymetry_kayak_data = "Nearshore Bathymetry - Kayak"
bathymetry_watercraft_data = "Nearshore Bathymetry - Personal Watercraft"
grainsize_data = "Surface-Sediment Grain-Size Distributions"
basemap_data = "Basemap"

elwha_data_types = [topography_data, grainsize_data]
# [topography_data]
# [topography_data, bathymetry_kayak_data, bathymetry_watercraft_data, grainsize_data]
# 
all_data_files = [file for data_type in elwha_data_types for file in os.listdir("data/Elwha/" + data_type)]

elwha_basemap_options = {
  "Default": basemaps.OpenStreetMap.Mapnik,
  "Satellite": basemaps.Esri.WorldImagery,
  "Topographic": basemaps.OpenTopoMap,
  "Black & White": basemaps.Stamen.Toner,
  "Dark": basemaps.CartoDB.DarkMatter
}

elwha = DataMap(
  map_center = (48.148, -123.553),
  basemap_options = elwha_basemap_options,
  data = all_data_files,
  legend = {
    "name": "Types of Data",
    "colors": {
      topography_data: "red",
      bathymetry_kayak_data: "yellow",
      bathymetry_watercraft_data: "green",
      grainsize_data: "#975411"
    }
  }
)

# -------------------------------------------------- Elwha Topo-Bathy Data Widgets --------------------------------------------------

basemap_select = pn.widgets.Select(name="Basemap", options=list(elwha_basemap_options.keys()))
elwha_data_multi_choice = pn.widgets.MultiChoice(name="Type of Data", value=[], options=elwha_data_types, solid=False)
data_date_range_slider = pn.widgets.DateRangeSlider(
  name="Data Collection Range",
  start=dt.datetime(2010, 9, 5), end=dt.datetime.utcnow(),
  value=(dt.datetime(2018, 1, 1), dt.datetime(2019, 1, 1)),
  bar_color="#0066cc"
)

# -------------------------------------------------- Helper Functions --------------------------------------------------

# Gets the point_style based on the data type.
def get_data_type_point_style(data_type):
  if data_type == topography_data:
    return {"color": "red", "opacity": 0.5, "radius": 0}     # simple dot for topography data since there's many datapoints for one latitude-longitude coordinate
  elif data_type == bathymetry_kayak_data:
    return {"color": "yellow", "radius": 0}
  elif data_type == bathymetry_watercraft_data:
    return {"color": "green", "radius": 0}
  elif data_type == grainsize_data:
    return {"color": "#975411", "opacity": 0.5, "fillColor": "#975411", "fillOpacity": 0.3, "radius": 8, "weight": 1, "dashArray": 2}
  return {}

# Gets the hover_style based on the data type.
def get_data_type_hover_style(data_type):
  if data_type == topography_data:
    return {"color": "#0066cc"}
  return {"color": "#0066cc", "fillColor": "#0066cc", "weight": 3}

# Gets info about the data file and creates a new GeoJSON layer with it.
def create_layer(file, data_type):
  print(file, data_type)
  # print("Loading data from " + file + "...")
  # Determine popup content based on different types of data.
  popup_info = {}
  if data_type == grainsize_data:
    popup_info = {
      "Date & Time Collected": [
        "Date Collected",
        {
          "Time (GMT)": "GMT",      # for grainsize data before July 2018
          "Time_GMT": "GMT"         # for grainsize data at and after July 2018
        }
      ],
      "Sample Type": ["Sample Type"],
      "Weight": ["Wt. percent in -2.00 phi bin", ["%"]],
      "Gravel": ["Percent Gravel", ["%"]],
      "Sand": ["Percent Sand", ["%"]],
      "Silt": ["Percent Silt", ["%"]],
      "Clay": ["Percent Clay", ["%"]],
      "Mud": ["Percent Mud", ["%"]]
    }
  elif (data_type == topography_data) or (data_type == bathymetry_kayak_data) or (data_type == bathymetry_watercraft_data):
    popup_info = {
      "Date & Time Collected": [
        {
          "Survey_Date": "",        # for topo-bathy data before July 2018
          "datetime_utc": "UTC"     # for topo-bathy data at and after July 2018
        }
      ],
      "Orthometric Height": [
        {
          "Ortho_Ht_m": "meters",
          "Ortho_ht_m": "meters",
          "ortho_ht_m": "meters"
        },
      ]
    }
  # Create and add GeoJSON layer to map.
  elwha.create_geojson_layer(
    data_path = base_data_path + data_type + "/" + file,
    layer_name = file,
    info = popup_info,
    longitude_col_names = ["Longitude", "longitude", "Longitude (deg. E)"],
    latitude_col_names = ["Latitude", "latitude", "Latitude (deg. N)"],
    point_style = get_data_type_point_style(data_type),
    hover_style = get_data_type_hover_style(data_type)
  )

# Filters data based on what data type(s) and time range that the user selects.
def filter_data_on_map(event):
  selected_data_types = elwha_data_multi_choice.value
  (selected_start_date, selected_end_date) = data_date_range_slider.value
  start_month, start_day, start_year = selected_start_date.month, selected_start_date.day, selected_start_date.year
  end_month, end_day, end_year = selected_end_date.month, selected_end_date.day, selected_end_date.year
  for data_type in elwha_data_types:
    data_type_files = os.listdir(base_data_path + data_type)
    for layer_name in data_type_files:
      if data_type in selected_data_types:
        # Create and add the selected data layer if it doesn't exist yet.
        if layer_name not in elwha.all_layers:
          print("create", layer_name)
          create_layer(layer_name, data_type)
        # Add the selected data layer if it isn't in map yet.
        elif elwha.all_layers[layer_name] not in elwha.map.layers:
          print("add", layer_name)
          elwha.add_geojson_layer(layer_name)
      # Else remove the data layer if user didn't select to display it and its data is in the map.
      elif (layer_name in elwha.all_layers) and (elwha.all_layers[layer_name] in elwha.map.layers):
        print("remove", layer_name)
        elwha.remove_geojson_layer(layer_name)

# -------------------------------------------------- Callbacks --------------------------------------------------

# Filter data whenever the selected data type(s) or date range change.
elwha_data_multi_choice.param.watch(filter_data_on_map, "value")
data_date_range_slider.param.watch(filter_data_on_map, "value")

# Update basemap whenever a different basemap value is selected.
basemap_select.param.watch(elwha.update_basemap, "value")

# -------------------------------------------------- Add Map Layers --------------------------------------------------

## Plot collected data as ipyleaflet layers.


In [ ]:
## Add components to serve on app.
elwha_data_visualizer = pn.Column(elwha.map, basemap_select, elwha_data_multi_choice, data_date_range_slider)
elwha_data_visualizer.servable()